# UFO Sightings

Building a web application to interactively predict the country where UFO was sighted, given duration of sight, city, longitude and latitude.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [1]:
!ls


UFO Sightings.ipynb complete.csv        scrubbed.csv


In [3]:
import pandas as pd
import numpy as np

ufos = pd.read_csv('./scrubbed.csv')
ufos.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (5,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [4]:
ufos.drop(columns = ["city","state","datetime", "shape", "duration (hours/min)", "comments", "date posted"], inplace = True )

In [5]:
ufos["seconds"] = ufos["duration (seconds)"]
ufos = ufos.drop(["duration (seconds)"], axis=1)

In [6]:
ufos.head()

,country,latitude,longitude,seconds
0,us,29.8830556,-97.941111,2700
1,NaN,29.38421,-98.581082,7200
2,gb,53.2,-2.916667,20
3,us,28.9783333,-96.645833,20
4,us,21.4180556,-157.803611,900


In [7]:
ufos.dropna(inplace=True)



In [11]:
ufos["seconds"].astype(str)

0          2700
2            20
3            20
4           900
5           300
          ...  
80327     600.0
80328    1200.0
80329    1200.0
80330       5.0
80331    1020.0
Name: seconds, Length: 70662, dtype: object

In [13]:
ufos["seconds"] = ufos["seconds"].apply(lambda x: str(x).replace("`", ""))
ufos["seconds"] = ufos["seconds"].astype(float)


In [17]:
ufos["seconds"].astype(int)

0        2700
2          20
3          20
4         900
5         300
         ... 
80327     600
80328    1200
80329    1200
80330       5
80331    1020
Name: seconds, Length: 70662, dtype: int64

In [19]:
ufos = ufos[(ufos['seconds'] >= 1) & (ufos['seconds'] <= 60)]

ufos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25863 entries, 2 to 80330
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   country     25863 non-null  object 
 1   latitude    25863 non-null  object 
 2   longitude   25863 non-null  float64
 3   seconds     25863 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1010.3+ KB


In [20]:
ufos.shape

(25863, 4)

In [21]:
ufos.head()

,country,latitude,longitude,seconds
2,gb,53.2,-2.916667,20.0
3,us,28.9783333,-96.645833,20.0
14,us,35.8238889,-80.253611,30.0
23,us,45.5827778,-122.352222,60.0
24,gb,51.783333,-0.783333,3.0


In [22]:
ufos.isnull().sum().sort_values(ascending=False)

country       0
latitude      0
longitude     0
seconds       0
dtype: int64

In [41]:
from sklearn.preprocessing import LabelEncoder

ufos['country'] = LabelEncoder().fit_transform(ufos['country'])


In [42]:
from sklearn.model_selection import train_test_split

Selected_features = ['seconds','latitude','longitude ']

X = ufos[Selected_features]
y = ufos['country']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [43]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

print(classification_report(y_test, predictions))
print('Predicted labels: ', predictions)
print('Accuracy: ', accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        41
           1       0.83      0.23      0.36       250
           2       1.00      1.00      1.00         8
           3       1.00      1.00      1.00       131
           4       0.96      1.00      0.98      4743

    accuracy                           0.96      5173
   macro avg       0.96      0.85      0.87      5173
weighted avg       0.96      0.96      0.95      5173

Predicted labels:  [4 4 4 ... 3 4 4]
Accuracy:  0.9605644693601392


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
import pickle
model_filename = 'ufo-model.pkl'
pickle.dump(model, open(model_filename,'wb'))



In [45]:
model = pickle.load(open('ufo-model.pkl','rb'))
print(model.predict([[50,44,-12]]))

[1]
